In [1]:
!pip install uproot
!pip install awkward
!pip install vector
!pip install tqdm

  Using cached cramjam-2.10.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------------------------- 0.0/375.7 kB ? eta -:--:--
   ----------- ---------------------------- 112.6/375.7 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 375.7/375.7 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/886.3 kB ? eta -:--:--
   --------------------------------------- 886.3/886.3 kB 27.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/536.5 kB ? eta -:--:--
   --------------------------------------- 536.5/536.5 kB 35.1 MB/s eta 0:00:00
Using cached cramjam-2.10.0-cp312-cp312-win_amd64.whl (1.7 MB)
Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl (30 kB)


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


  Using cached vector-1.6.2-py3-none-any.whl.metadata (15 kB)
Using cached vector-1.6.2-py3-none-any.whl (177 kB)


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


16 --> exotic signal
43 --> nominal higgs signal

In [2]:
# Import basic libraries
import copy # copy variables
import os   # manage paths

import uproot   # use of root files
import awkward as ak    # nested, variable sized data
import vector   # lorentz vectors
vector.register_awkward()
import matplotlib.pyplot as plt # plotting
import tqdm # progress bars

In [6]:
filename1 = "WToQQ_120.root"
filename2 = "TTBar_120.root"

In [7]:

print('TTree objects inside the ROOT file:')
for ii in uproot.open(filename1).keys():
    print('-',ii)

for ii in uproot.open(filename2).keys():
    print('-',ii)



TTree objects inside the ROOT file:
- tree;1
- tree;1


In [15]:
tree = uproot.open({filename: "tree;1"})

In [16]:
# Display only the first 10 branches
first_10_branches = list(tree.keys())[:100]
tree.show(filter_name=first_10_branches, name_width=50, typename_width=50, interpretation_width=50)

name                                               | typename                                           | interpretation                                    
---------------------------------------------------+----------------------------------------------------+---------------------------------------------------
part_px                                            | std::vector<float>                                 | AsJagged(AsDtype('>f4'), header_bytes=10)
part_py                                            | std::vector<float>                                 | AsJagged(AsDtype('>f4'), header_bytes=10)
part_pz                                            | std::vector<float>                                 | AsJagged(AsDtype('>f4'), header_bytes=10)
part_energy                                        | std::vector<float>                                 | AsJagged(AsDtype('>f4'), header_bytes=10)
part_deta                                          | std::vector<float>                       

In [1]:
import uproot
import json
import numpy as np

def convert_root_to_json(root_file_path, json_file_path, max_events=2000):
    with uproot.open(root_file_path) as file:
        tree = file["tree"]  # Usamos "tree" según tu descripción
        branches = tree.keys()
        arrays = tree.arrays(branches, entry_stop=max_events)

        data = []
        num_events = len(arrays["jet_pt"])  # Usamos una rama escalar para contar eventos

        for i in range(num_events):
            event = {}
            for branch in branches:
                value = arrays[branch][i]
                # Convertimos listas de partículas o valores escalares
                if isinstance(value, np.ndarray):
                    event[branch] = value.tolist()
                elif hasattr(value, "tolist"):  # tipos NumPy escalares
                    event[branch] = value.tolist()
                else:
                    event[branch] = value
            data.append(event)

        with open(json_file_path, "w") as f:
            json.dump(data, f, indent=4)

        print(f"✅ Archivo convertido exitosamente a '{json_file_path}'")

# Uso del script
if __name__ == "__main__":
    convert_root_to_json("WToQQ_120.root", "WToQQ_120.json", max_events=10000)
    convert_root_to_json("TTBar_120.root", "TTBar_120.json", max_events=10000)
    convert_root_to_json("ZJetsToNuNu_000.root", "ZJetsToNuNu_000.json", max_events=10000)

✅ Archivo convertido exitosamente a 'WToQQ_120.json'
✅ Archivo convertido exitosamente a 'TTBar_120.json'
✅ Archivo convertido exitosamente a 'ZJetsToNuNu_000.json'


In [20]:
jets = ak.zip(
     {
        "pt": tree["AnalysisJetsAuxDyn.pt"].array(),
        "eta": tree["AnalysisJetsAuxDyn.eta"].array(),
        "phi": tree["AnalysisJetsAuxDyn.phi"].array(),
    }
)

In [21]:
jets

<Array [[{pt: 8.86e+04, ...}, ..., {...}], ...] type='120000 * var * {pt: f...'>

In [12]:
jets_as_list = ak.to_list(jets) 

https://opendata.cern.ch/record/80012

In [ ]:
schema = {
    "Jets": [
        "pt", "eta", "phi"
    ]
}

def read_events(filename, schema):
    with uproot.open({filename: "CollectionTree"}) as tree:
        events = {}
        for objname, fields in schema.items():
            base = objname
            if objname in ["Jets"]:
                base = "Analysis" + objname
                ak_zip = vector.zip
            else:
                ak_zip = ak.zip
            arrays = tree.arrays(
                fields,
                aliases={field: f"{base}AuxDyn.{field}" for field in fields},
            )
            arrays = ak_zip(dict(zip(arrays.fields, ak.unzip(arrays))))
            events[objname] = arrays
        return ak.zip(events, depth_limit=1)


In [14]:
import json

with open("jets_data_higgs_nominal_samples.json", "w") as f:
    json.dump(jets_as_list, f, indent=2)

In [16]:
filename = "DAOD_PHYSLITE.38191516._000001.pool.root.1"
print('TTree objects inside the ROOT file:')
for ii in uproot.open(filename).keys():
    print('-',ii)
tree = uproot.open({filename: "CollectionTree"})
# Display only the first 10 branches
first_10_branches = list(tree.keys())[:10]
tree.show(filter_name=first_10_branches, name_width=50, typename_width=50, interpretation_width=50)
jets = ak.zip(
     {
        "pt": tree["AnalysisJetsAuxDyn.pt"].array(),
        "eta": tree["AnalysisJetsAuxDyn.eta"].array(),
        "phi": tree["AnalysisJetsAuxDyn.phi"].array(),
    }
)
jets_as_list = ak.to_list(jets) 
with open("jets_data_exotic_signals.json", "w") as f:
    json.dump(jets_as_list, f, indent=2)

TTree objects inside the ROOT file:
- ##Params;3
- ##Params;2
- ##Shapes;3
- ##Shapes;2
- ##Links;3
- ##Links;2
- MetaData;1
- MetaDataHdr;1
- MetaDataHdrForm;1
- CollectionTree;1
- POOLContainer;1
- POOLContainerForm;1
- POOLCollectionTree;1
name                                               | typename                                           | interpretation                                    
---------------------------------------------------+----------------------------------------------------+---------------------------------------------------
xTrigDecisionAux.                                  | xAOD::TrigDecisionAuxInfo_v1                       | AsGroup(<TBranchElement 'xTrigDecisionAux.' (20...
xTrigDecisionAux./xTrigDecisionAux.xAOD::AuxInf... | unknown                                            | <UnknownInterpretation 'none of the rules match...
xTrigDecisionAux./xTrigDecisionAux.smk             | uint32_t                                           | AsDtype('>u4')
xTrigDec